In [4]:
import requests
import pandas as pd
from env import api_key
from acquire import acquire
from datetime import timedelta, datetime
from symbols_to_track import list_of_symbols

In [17]:
url = "https://yfapi.net/v6/finance/quote"

querystring = {
    "symbols":"AAPL,MSFT,TSLA"
    }

headers = {
    'x-api-key': api_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [18]:
data = response.json()

In [19]:
d = data['quoteResponse']['result'][0]

In [24]:
example = pd.DataFrame()

In [25]:
example = example.append(d, ignore_index=True)

In [31]:
pd.set_option("display.max_rows", None, "display.max_columns", None) 

pd.reset_option("display.max_rows", "display.max_columns")

In [26]:
example.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 72 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ask                                1 non-null      float64
 1   askSize                            1 non-null      float64
 2   averageAnalystRating               1 non-null      object 
 3   averageDailyVolume10Day            1 non-null      float64
 4   averageDailyVolume3Month           1 non-null      float64
 5   bid                                1 non-null      float64
 6   bidSize                            1 non-null      float64
 7   bookValue                          1 non-null      float64
 8   currency                           1 non-null      object 
 9   displayName                        1 non-null      object 
 10  dividendDate                       1 non-null      float64
 11  earningsTimestamp                  1 non-null      float64
 12

In [27]:
example

,ask,askSize,averageAnalystRating,averageDailyVolume10Day,averageDailyVolume3Month,bid,bidSize,bookValue,currency,displayName,...,sourceInterval,symbol,tradeable,trailingAnnualDividendRate,trailingAnnualDividendYield,trailingPE,triggerable,twoHundredDayAverage,twoHundredDayAverageChange,twoHundredDayAverageChangePercent
0,174.15,12.0,1.8 - Buy,85371860.0,91216271.0,174.12,11.0,3.841,USD,Apple,...,15.0,AAPL,0.0,0.85,0.004936,31.087843,1.0,145.57954,28.823257,0.19799


In [12]:
def get_daily_data():
    '''
    This functions brings all other functions together
    First we grab all the top gainers for the day callign get_day_gainers
    Checks to see how many total stocks there are 
    API can get prices of 10 total per request so we bundle them up by 10
    Request 10 at time and load them into the main dataframe df
    return df
    '''
    # Set headers to include my api_key
    headers = {
        'x-api-key': api_key
        }
    
    # Create an emtpy DataFrame
    df = pd.DataFrame()

    # Check the size of the todays top gainers list
    total = len(list_of_symbols)

    # Create a variable named end to be where the range will end
    end = ((total//10)*10)+10

    # Check to see if total is evenly divisible by 10.  This is because api can grab 10 symbols per request
    if total%10 !=0:

        # If the total is not evenly divisible by 10, loop through list by 10s, then grab the ones that are left later
        for y in range(10, end, 10):
            
            # Setting x to be 10 less than y
            x = y-10
            
            # Creating a string called placeholder to be from range x:y (groups of 10) in the long list of symbols
            placeholder = create_placeholder(list_of_symbols[x:y])
            
            querystring = {
                "symbols":f"{placeholder}"
                }
            
            # Grabbing daily information for ten stocks at a time
            response = requests.request("GET", "https://yfapi.net/v6/finance/quote", headers=headers, params=querystring)
            
            # Save the response to data
            data = response.json()
            
            # Creating an empty temporary Data Frame
            temp = pd.DataFrame()

            # Extracting the prices from json file
            temp = extract_all(data)

            # Adding the 10 stocks last 5 years to the main DataFrame df
            df = pd.concat([df, temp], axis = 1)
        
        # Clearing temp
        temp = pd.DataFrame()

        # Grabbing the rest, should be less than 10 stocks symbols left
        placeholder = create_placeholder(list_of_symbols[end-10:total])
        
        querystring = {
            "symbols":f"{placeholder}"
            }
        # Repeating the process to get the stock information for the last remaining symbols
        response = requests.request("GET", "https://yfapi.net/v6/finance/quote", headers=headers, params=querystring)
        
        # Saving response to data again
        data = response.json()

        # Extracting once more
        temp = extract_all(data)

        # Adding the final symbols to the main dataframe df
        df = pd.concat([df, temp], axis = 0)

    else:

        # If total is evenly divisible by 10, no need to add the extra step of getting the rest
        # Just Loop by 10
        # Setting x to be 10 less than y
        for y in range(10, end, 10):
            x = y-10
            
            # Creating a string called placeholder to be from range x:y (groups of 10) in the long list of symbols
            placeholder = create_placeholder(list_of_symbols[x:y])
            
            querystring = {
                "symbols":f"{placeholder}"
                }
            
            # Grabbing daily information for ten stocks at a time
            response = requests.request("GET", "https://yfapi.net/v6/finance/quote", headers=headers, params=querystring)
            
            # Save the response to data
            data = response.json()
            
            # Creating an empty temporary Data Frame
            temp = pd.DataFrame()

            # Extracting the prices from json file
            temp = extract_all(data)

            # Adding the 10 stocks last 5 years to the main DataFrame df
            df = pd.concat([df, temp], axis = 0)

    return df

In [36]:
data

{'quoteResponse': {'result': [{'language': 'en-US',
    'region': 'US',
    'quoteType': 'EQUITY',
    'quoteSourceName': 'Nasdaq Real Time Price',
    'triggerable': True,
    'currency': 'USD',
    'regularMarketChange': 2.2789917,
    'regularMarketChangePercent': 1.3235332,
    'regularMarketTime': 1641926168,
    'regularMarketPrice': 174.469,
    'regularMarketDayHigh': 175.03,
    'regularMarketDayRange': '170.83 - 175.03',
    'regularMarketDayLow': 170.83,
    'regularMarketVolume': 48558122,
    'regularMarketPreviousClose': 172.19,
    'bid': 174.64,
    'ask': 174.63,
    'bidSize': 10,
    'askSize': 8,
    'fullExchangeName': 'NasdaqGS',
    'financialCurrency': 'USD',
    'regularMarketOpen': 172.32,
    'averageDailyVolume3Month': 91216271,
    'averageDailyVolume10Day': 85371860,
    'fiftyTwoWeekLowChange': 58.258995,
    'fiftyTwoWeekLowChangePercent': 0.50132513,
    'fiftyTwoWeekRange': '116.21 - 182.94',
    'fiftyTwoWeekHighChange': -8.471008,
    'fiftyTwoWeekHi

In [43]:
data['quoteResponse']['result']

[{'language': 'en-US',
  'region': 'US',
  'quoteType': 'EQUITY',
  'quoteSourceName': 'Nasdaq Real Time Price',
  'triggerable': True,
  'currency': 'USD',
  'regularMarketChange': 2.2789917,
  'regularMarketChangePercent': 1.3235332,
  'regularMarketTime': 1641926168,
  'regularMarketPrice': 174.469,
  'regularMarketDayHigh': 175.03,
  'regularMarketDayRange': '170.83 - 175.03',
  'regularMarketDayLow': 170.83,
  'regularMarketVolume': 48558122,
  'regularMarketPreviousClose': 172.19,
  'bid': 174.64,
  'ask': 174.63,
  'bidSize': 10,
  'askSize': 8,
  'fullExchangeName': 'NasdaqGS',
  'financialCurrency': 'USD',
  'regularMarketOpen': 172.32,
  'averageDailyVolume3Month': 91216271,
  'averageDailyVolume10Day': 85371860,
  'fiftyTwoWeekLowChange': 58.258995,
  'fiftyTwoWeekLowChangePercent': 0.50132513,
  'fiftyTwoWeekRange': '116.21 - 182.94',
  'fiftyTwoWeekHighChange': -8.471008,
  'fiftyTwoWeekHighChangePercent': -0.046304844,
  'fiftyTwoWeekLow': 116.21,
  'fiftyTwoWeekHigh': 18

In [52]:
df = pd.DataFrame()

In [55]:
for symbol in data['quoteResponse']['result']:
    temp = pd.DataFrame()
    temp = temp.append(symbol, ignore_index=True)
    df = df.append(temp, ignore_index=True)

In [56]:
df

,ask,askSize,averageAnalystRating,averageDailyVolume10Day,averageDailyVolume3Month,bid,bidSize,bookValue,currency,displayName,dividendDate,earningsTimestamp,earningsTimestampEnd,earningsTimestampStart,epsCurrentYear,epsForward,epsTrailingTwelveMonths,esgPopulated,exchange,exchangeDataDelayedBy,exchangeTimezoneName,exchangeTimezoneShortName,fiftyDayAverage,fiftyDayAverageChange,fiftyDayAverageChangePercent,fiftyTwoWeekHigh,fiftyTwoWeekHighChange,fiftyTwoWeekHighChangePercent,fiftyTwoWeekLow,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,fiftyTwoWeekRange,financialCurrency,firstTradeDateMilliseconds,forwardPE,fullExchangeName,gmtOffSetMilliseconds,language,longName,market,marketCap,marketState,messageBoardId,pageViewGrowthWeekly,priceEpsCurrentYear,priceHint,priceToBook,quoteSourceName,quoteType,region,regularMarketChange,regularMarketChangePercent,regularMarketDayHigh,regularMarketDayLow,regularMarketDayRange,regularMarketOpen,regularMarketPreviousClose,regularMarketPrice,regularMarketTime,regularMarketVolume,sharesOutstanding,shortName,sourceInterval,symbol,tradeable,trailingAnnualDividendRate,trailingAnnualDividendYield,trailingPE,triggerable,twoHundredDayAverage,twoHundredDayAverageChange,twoHundredDayAverageChangePercent
0,174.63,8.0,1.8 - Buy,85371860.0,91216271.0,174.64,10.0,3.841,USD,Apple,1.636589e+09,1.643317e+09,1.643317e+09,1.643317e+09,5.74,6.18,5.610,0.0,NMS,0.0,America/New_York,EST,165.2064,9.262588,0.056067,182.94,-8.471008,-0.046305,116.21,58.258995,0.501325,116.21 - 182.94,USD,3.454794e+11,28.231230,NasdaqGS,-18000000.0,en-US,Apple Inc.,us_market,2.849846e+12,REGULAR,finmb_24937,-0.108297,30.395296,2.0,45.422806,Nasdaq Real Time Price,EQUITY,US,2.278992,1.323533,175.03,170.83,170.83 - 175.03,172.32,172.19,174.469,1.641926e+09,48558122.0,1.633440e+10,Apple Inc.,15.0,AAPL,0.0,0.85,0.004936,31.099642,1.0,145.57954,28.889450,0.198444
1,315.13,9.0,1.7 - Buy,28284290.0,26956895.0,315.11,10.0,20.242,USD,Microsoft,1.646870e+09,1.635265e+09,1.643371e+09,1.643022e+09,9.22,10.55,8.939,0.0,NMS,0.0,America/New_York,EST,332.5802,-17.410187,-0.052349,349.67,-34.500000,-0.098664,212.03,103.140015,0.486441,212.03 - 349.67,USD,5.111082e+11,29.873934,NasdaqGS,-18000000.0,en-US,Microsoft Corporation,us_market,2.366290e+12,REGULAR,finmb_21835,0.443205,34.183296,2.0,15.570102,Nasdaq Real Time Price,EQUITY,US,0.900024,0.286386,316.61,309.89,309.89 - 316.61,313.38,314.27,315.170,1.641926e+09,16184290.0,7.507980e+09,Microsoft Corporation,15.0,MSFT,0.0,2.30,0.007319,35.257860,1.0,290.04474,25.125275,0.086626
2,1065.44,14.0,2.6 - Hold,25117920.0,27616271.0,1064.09,22.0,26.945,USD,Tesla,NaN,1.634746e+09,1.643663e+09,1.643144e+09,6.26,8.82,3.062,0.0,NMS,0.0,America/New_York,EST,1073.0153,-9.505249,-0.008858,1243.49,-179.979980,-0.144738,539.49,524.020000,0.971325,539.49 - 1243.49,USD,1.277818e+12,120.579370,NasdaqGS,-18000000.0,en-US,"Tesla, Inc.",us_market,1.068041e+12,REGULAR,finmb_27444752,-0.001168,169.889770,2.0,39.469660,Nasdaq Real Time Price,EQUITY,US,5.390015,0.509395,1075.75,1038.82,1038.82 - 1075.75,1053.67,1058.12,1063.510,1.641926e+09,16215322.0,1.004260e+09,"Tesla, Inc.",15.0,TSLA,0.0,NaN,NaN,347.325300,1.0,795.58190,267.928100,0.336770


In [13]:
def extract_all(data):
    '''
    This function takes in the quote response and extracts all the information for each symbol.
    '''
    
    # Create the Main Data Frame
    df = pd.DataFrame()
    
    # Loop through data
    for symbol in data['quoteResponse']['result']:
        
        # Create an empty temporary Data Frame
        temp = pd.DataFrame()
        
        # Save to temp
        temp = temp.append(symbol, ignore_index=True)
        
        # Save to Main Data Frame
        df = df.append(temp, ignore_index=True)
        
    return df

In [15]:
df = get_daily_data()

In [16]:
df

,ask,askSize,averageAnalystRating,averageDailyVolume10Day,averageDailyVolume3Month,bid,bidSize,bookValue,currency,displayName,...,twoHundredDayAverageChangePercent,dividendDate,trailingAnnualDividendRate,trailingAnnualDividendYield,ipoExpectedDate,nameChangeDate,prevName,trailingThreeMonthNavReturns,trailingThreeMonthReturns,ytdReturn
0,137.37,10.0,2.3 - Buy,61656660.0,57689504.0,137.26,13.0,5.888,USD,Advanced Micro Devices,...,0.289353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,174.08,12.0,1.8 - Buy,85371860.0,91216271.0,174.06,11.0,3.841,USD,Apple,...,0.197906,1.636589e+09,0.85,0.004936,NaN,NaN,NaN,NaN,NaN,NaN
2,1062.79,14.0,2.6 - Hold,25117920.0,27616271.0,1061.82,22.0,26.945,USD,Tesla,...,0.333100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.00,18.0,3.9 - Underperform,37342120.0,42902607.0,0.00,8.0,-3.200,USD,AMC Entertainment,...,-0.330088,1.584922e+09,0.03,0.001317,NaN,NaN,NaN,NaN,NaN,NaN
4,0.00,9.0,2.5 - Buy,127419850.0,101454804.0,0.00,14.0,9.157,USD,NaN,...,0.573460,1.638317e+09,0.00,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,177.25,12.0,2.3 - Buy,3543880.0,4324753.0,176.92,11.0,17.164,USD,Zoom Video Communications,...,-0.405948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.00,280.0,3.8 - Underperform,5178380.0,5124879.0,0.00,215.0,3.299,USD,Invesco Mortgage Capital,...,-0.143094,1.643242e+09,0.35,0.121528,NaN,NaN,NaN,NaN,NaN,NaN
7,0.00,11.0,1.7 - Buy,4282760.0,4364903.0,0.00,29.0,11.346,USD,New Residential Inv,...,0.032195,1.643328e+09,0.85,0.078053,NaN,NaN,NaN,NaN,NaN,NaN
8,0.00,10.0,2.1 - Buy,9069260.0,7939887.0,0.00,32.0,4.140,USD,fuboTV,...,-0.386557,1.551312e+09,NaN,NaN,NaN,2022-01-11,"Facebank Group, Inc.",NaN,NaN,NaN


In [14]:
def create_placeholder(list_of_symbols):
    '''
    This function will take in a list of symbols
    Creates a single string with no spaces only commas
    Returns thats string
    '''
    
    # Creating an empty string
    placeholder = ""

    # Creating a placeholder string from list of symbols
    for index, symbol in enumerate(list_of_symbols):

        # if index is equal to nine then this is last symbol and doesn't require a comma at the end
        if index == 9:
            placeholder += symbol

        # All other index's will require a comma
        else:
            placeholder += symbol + ","

    return placeholder